In [46]:
import py_vollib.black_scholes.greeks.analytical as greeks
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility
import pandas_datareader.data as web
import pandas as pd 
import datetime

In [47]:


def get_greeks(option_price: float, S: float, X: float, days_till_exp: float, option_type: str) -> pd.DataFrame:
    # Get today's 1-year Treasury Rate
    today = datetime.datetime.today()
    yesterday = today - datetime.timedelta(days=3)  # Needs to be 3 to account for weekends, if used on weekends
    treasury_rate_1yr = web.DataReader('DGS1', 'fred', yesterday, today).iloc[-1].values[0]
    
    T = (days_till_exp / 365)
    r = (treasury_rate_1yr / 100)  # Risk-free rate (annual)
    sigma = implied_volatility(option_price, S, X, T, r, 0, option_type)  # Volatility (annual)

    # Calculate the Greeks and return them as a DataFrame
    return pd.DataFrame(
        {
            'Delta': [greeks.delta(option_type, S, X, T, r, sigma)],
            'Gamma': [greeks.gamma(option_type, S, X, T, r, sigma)],
            'Vega': [greeks.vega(option_type, S, X, T, r, sigma)],
            'Theta': [greeks.theta(option_type, S, X, T, r, sigma)],
            'Rho': [greeks.rho(option_type, S, X, T, r, sigma)], 
            'Implied_vol (%)': [sigma*100]
        }
    )

In [48]:
option_type = 'p'

price = 0.41
S = 3.09  # Current price of the underlying asset
X = 2.50  # Strike price of the option
T = 20  # Time to maturity in years


In [49]:
get_greeks(price, S, X, T, option_type).round(5)

,Delta,Gamma,Vega,Theta,Rho,Implied_vol (%)
0,-0.25581,0.17078,0.00233,-0.015,-0.00066,260.38447
